# 🚀 Football Prediction - Google Colab Training

**Complete ensemble training pipeline for Google Colab**

✅ Auto-downloads data (no API key needed)
✅ Trains 4 models (XGBoost, LightGBM, CatBoost, Neural Net)
✅ Saves to Google Drive
✅ One-click download

**Instructions:**
1. Runtime → Change runtime type → GPU (recommended)
2. Run all cells
3. Download models or save to Drive

In [ ]:
# Install dependencies
!pip install -q xgboost lightgbm catboost torch onnx onnxruntime

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'🖥️ Using: {DEVICE}')

## 1. Download Training Data (No API Key Needed!)

In [ ]:
# Download from GitHub (45,000+ international matches)
url = 'https://raw.githubusercontent.com/martj42/international_results/master/results.csv'
df = pd.read_csv(url)
print(f'✅ Loaded {len(df):,} matches from 1872-2024')
df.head()

## 2. Feature Engineering

In [ ]:
class FeatureEngineer:
    def __init__(self, K=32):
        self.elo = {}
        self.K = K
        self.team_encoder = LabelEncoder()
    
    def get_elo(self, team):
        return self.elo.get(team, 1500)
    
    def update_elo(self, home, away, result):
        h_elo, a_elo = self.get_elo(home), self.get_elo(away)
        exp_h = 1 / (1 + 10**((a_elo - h_elo) / 400))
        if result == 'H': s_h, s_a = 1, 0
        elif result == 'A': s_h, s_a = 0, 1
        else: s_h, s_a = 0.5, 0.5
        self.elo[home] = h_elo + self.K * (s_h - exp_h)
        self.elo[away] = a_elo + self.K * (s_a - (1 - exp_h))
    
    def process(self, df):
        df = df.copy()
        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values('date').reset_index(drop=True)
        
        # Result column
        df['result'] = np.where(df['home_score'] > df['away_score'], 'H',
                                np.where(df['home_score'] < df['away_score'], 'A', 'D'))
        
        # Calculate Elo for each match
        print('📊 Calculating Elo ratings...')
        elo_h, elo_a, elo_diff = [], [], []
        for _, row in df.iterrows():
            h, a = self.get_elo(row['home_team']), self.get_elo(row['away_team'])
            elo_h.append(h); elo_a.append(a); elo_diff.append(h - a)
            self.update_elo(row['home_team'], row['away_team'], row['result'])
        
        df['home_elo'] = elo_h
        df['away_elo'] = elo_a
        df['elo_diff'] = elo_diff
        
        # Encode teams
        all_teams = pd.concat([df['home_team'], df['away_team']]).unique()
        self.team_encoder.fit(all_teams)
        df['home_enc'] = self.team_encoder.transform(df['home_team'])
        df['away_enc'] = self.team_encoder.transform(df['away_team'])
        
        # Date features
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
        df['dow'] = df['date'].dt.dayofweek
        
        print(f'✅ Processed {len(df):,} matches with {len(self.elo):,} teams')
        return df

fe = FeatureEngineer()
df = fe.process(df)

# Show top teams by Elo
top_teams = sorted(fe.elo.items(), key=lambda x: x[1], reverse=True)[:10]
print('\n🏆 Top 10 Teams by Elo:')
for i, (team, elo) in enumerate(top_teams, 1):
    print(f'  {i}. {team}: {elo:.0f}')

## 3. Prepare Training Data

In [ ]:
# Features and target
FEATURES = ['home_enc', 'away_enc', 'home_elo', 'away_elo', 'elo_diff', 'year', 'month', 'dow']

le_result = LabelEncoder()
df['result_enc'] = le_result.fit_transform(df['result'])

X = df[FEATURES].values
y = df['result_enc'].values

# Time-based split (train on past, test on recent)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

print(f'📊 Training set: {len(X_train):,} matches')
print(f'📊 Test set: {len(X_test):,} matches')
print(f'📊 Classes: {list(le_result.classes_)}')

## 4. Train Ensemble Models

In [ ]:
models = {}

print('🚀 Training XGBoost...')
xgb = XGBClassifier(n_estimators=300, max_depth=8, learning_rate=0.05, 
                    random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(X_train, y_train)
models['xgb'] = xgb
xgb_acc = accuracy_score(y_test, xgb.predict(X_test))
print(f'   ✅ XGBoost Accuracy: {xgb_acc:.4f}')

print('🚀 Training LightGBM...')
lgb = LGBMClassifier(n_estimators=300, max_depth=8, learning_rate=0.05, random_state=42, verbose=-1)
lgb.fit(X_train, y_train)
models['lgb'] = lgb
lgb_acc = accuracy_score(y_test, lgb.predict(X_test))
print(f'   ✅ LightGBM Accuracy: {lgb_acc:.4f}')

print('🚀 Training CatBoost...')
cat = CatBoostClassifier(iterations=300, depth=8, learning_rate=0.05, random_state=42, verbose=0)
cat.fit(X_train, y_train)
models['cat'] = cat
cat_acc = accuracy_score(y_test, cat.predict(X_test))
print(f'   ✅ CatBoost Accuracy: {cat_acc:.4f}')

## 5. Train Neural Network

In [ ]:
class FootballNet(nn.Module):
    def __init__(self, input_dim, hidden=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(hidden, 64),
            nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(64, 3)
        )
    def forward(self, x):
        return self.net(x)

# Normalize features
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# Train
print('🚀 Training Neural Network...')
net = FootballNet(len(FEATURES)).to(DEVICE)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

X_t = torch.FloatTensor(X_train_s).to(DEVICE)
y_t = torch.LongTensor(y_train).to(DEVICE)

for epoch in range(100):
    net.train()
    optimizer.zero_grad()
    loss = criterion(net(X_t), y_t)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 25 == 0:
        print(f'   Epoch {epoch+1}/100, Loss: {loss.item():.4f}')

net.eval()
with torch.no_grad():
    nn_preds = net(torch.FloatTensor(X_test_s).to(DEVICE)).argmax(1).cpu().numpy()
nn_acc = accuracy_score(y_test, nn_preds)
print(f'   ✅ Neural Net Accuracy: {nn_acc:.4f}')

## 6. Ensemble Prediction

In [ ]:
WEIGHTS = {'xgb': 0.30, 'lgb': 0.30, 'cat': 0.25, 'nn': 0.15}

def ensemble_predict(X, X_scaled, models, net, weights=WEIGHTS):
    probs = np.zeros((len(X), 3))
    probs += weights['xgb'] * models['xgb'].predict_proba(X)
    probs += weights['lgb'] * models['lgb'].predict_proba(X)
    probs += weights['cat'] * models['cat'].predict_proba(X)
    
    net.eval()
    with torch.no_grad():
        nn_probs = torch.softmax(net(torch.FloatTensor(X_scaled).to(DEVICE)), dim=1).cpu().numpy()
    probs += weights['nn'] * nn_probs
    return probs

ens_probs = ensemble_predict(X_test, X_test_s, models, net)
ens_preds = ens_probs.argmax(1)
ens_acc = accuracy_score(y_test, ens_preds)

print('\n🏆 FINAL RESULTS:')
print(f'   XGBoost:    {xgb_acc:.4f}')
print(f'   LightGBM:   {lgb_acc:.4f}')
print(f'   CatBoost:   {cat_acc:.4f}')
print(f'   Neural Net: {nn_acc:.4f}')
print(f'   ⭐ ENSEMBLE: {ens_acc:.4f}')

print('\n📊 Classification Report:')
print(classification_report(y_test, ens_preds, target_names=le_result.classes_))

## 7. Export Models

In [ ]:
import pickle
import json
import os

# Create output directory
os.makedirs('trained_models', exist_ok=True)

# Save models
models['xgb'].save_model('trained_models/xgb_football.json')
models['lgb'].booster_.save_model('trained_models/lgb_football.txt')
models['cat'].save_model('trained_models/cat_football.cbm')
torch.save(net.state_dict(), 'trained_models/nn_football.pt')

# Save encoders and config
with open('trained_models/encoders.pkl', 'wb') as f:
    pickle.dump({'team_enc': fe.team_encoder, 'result_enc': le_result, 'scaler': scaler}, f)

with open('trained_models/elo_ratings.json', 'w') as f:
    json.dump(fe.elo, f)

with open('trained_models/model_meta.json', 'w') as f:
    json.dump({
        'features': FEATURES,
        'classes': list(le_result.classes_),
        'ensemble_weights': WEIGHTS,
        'accuracy': float(ens_acc),
        'num_teams': len(fe.elo)
    }, f, indent=2)

print('✅ All models saved to trained_models/')
!ls -la trained_models/

## 8. Download Models

In [ ]:
# Create ZIP file for easy download
!cd trained_models && zip -r ../football_models.zip .

from google.colab import files
files.download('football_models.zip')

print('\n📥 Download complete! Extract to your Flask app:')
print('   - xgb_football.json → models/trained/')
print('   - lgb_football.txt → models/trained/')
print('   - cat_football.cbm → models/trained/')
print('   - nn_football.pt → models/trained/')
print('   - encoders.pkl → models/config/')
print('   - elo_ratings.json → models/config/')
print('   - model_meta.json → models/config/')

## 9. (Optional) Save to Google Drive

In [ ]:
# Uncomment to save to Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r trained_models /content/drive/MyDrive/football_models
# print('✅ Saved to Google Drive!')

## 10. Test Prediction

In [ ]:
def predict_match(home_team, away_team):
    # Get Elo
    h_elo = fe.elo.get(home_team, 1500)
    a_elo = fe.elo.get(away_team, 1500)
    
    # Encode teams
    try:
        h_enc = fe.team_encoder.transform([home_team])[0]
        a_enc = fe.team_encoder.transform([away_team])[0]
    except:
        h_enc, a_enc = 0, 0
    
    # Features
    import datetime
    now = datetime.datetime.now()
    features = np.array([[h_enc, a_enc, h_elo, a_elo, h_elo-a_elo, now.year, now.month, now.weekday()]])
    
    # Ensemble prediction
    probs = ensemble_predict(features, scaler.transform(features), models, net)
    
    # Results
    classes = le_result.classes_
    pred_idx = probs[0].argmax()
    
    print(f'\n⚽ {home_team} vs {away_team}')
    print(f'   Elo: {h_elo:.0f} vs {a_elo:.0f}')
    print(f'   Home Win: {probs[0][list(classes).index("H")]*100:.1f}%')
    print(f'   Draw: {probs[0][list(classes).index("D")]*100:.1f}%')
    print(f'   Away Win: {probs[0][list(classes).index("A")]*100:.1f}%')
    print(f'   🏆 Prediction: {"Home Win" if classes[pred_idx]=="H" else "Away Win" if classes[pred_idx]=="A" else "Draw"}')

# Test predictions
predict_match('Germany', 'Brazil')
predict_match('Argentina', 'France')
predict_match('England', 'Spain')